<a href="https://colab.research.google.com/github/tam1444AH/UH-Insure-NSA/blob/main/notebooks/codeLLMFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
%pip install nbstripout

REPO_URL="https://github.com/tam1444AH/UH-Insure-NSA.git"
REPO="UH-Insure-NSA"

os.chdir("/content")

# If repo exists, update it; otherwise, clone fresh
if os.path.exists(REPO):
    print(f"Repo '{REPO}' exists, pulling latest changes...")
    os.chdir(REPO)
    !git reset --hard HEAD   # optional: discard local changes
    !git pull
else:
    print(f"Cloning repo '{REPO}'...")
    !git clone "$REPO_URL" "$REPO"
    os.chdir(REPO)

!nbstripout --install
!git branch -a


# Install dependencies if present
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt
if os.path.exists("pyproject.toml"):
    !pip install -e .

Cloning repo 'UH-Insure-NSA'...
Cloning into 'UH-Insure-NSA'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 388 (delta 89), reused 12 (delta 12), pack-reused 243 (from 2)
Receiving objects: 100% (388/388), 599.24 KiB | 15.36 MiB/s, done.
Resolving deltas: 100% (216/216), done.
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 68.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.8 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [ ]:
from model.test import test
test()

Hello World!


In [ ]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

Logged in as: tam2003


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
os.chdir("/content")
!pip install -q transformers datasets

import json, random
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer

In [ ]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen3-Coder-30B-A3B-Instruct"  # Model checkpoint on the Hugging Face Hub
DATASET = "/content/unsupervised_hybrid_data.jsonl" # Josh's preproccesed dataset.
DATA_COLUMN = "content"  # Column name containing the code content

SEQ_LENGTH = 2048  # Sequence length

# Training arguments
MAX_STEPS = 200 #2000  # max_steps
BATCH_SIZE = 12  # batch_size
GR_ACC_STEPS = 2  # gradient_accumulation_steps
LR = 2e-4                     # learning_rate
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.05  # weight_decay
NUM_WARMUP_STEPS = 15  # num_warmup_steps
EVAL_FREQ = 25
SAVE_FREQ = 50
LOG_FREQ = 10
OUTPUT_DIR = "peft-FT-3-Coder-30b"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.25  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate


# LORA
LORA_R = 16  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = "c_proj,c_attn,q_attn,c_fc,c_proj"  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 0

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)

set_seed(SEED)

In [ ]:
import torch
from tqdm import tqdm

SRC = "/content/unsupervised_hybrid_data.jsonl"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

def detect_type(filename):
  if filename.endswith(".cry"):
    return "cryptol"
  elif filename.endswith(".saw"):
    return "saw"
  else:
    return "text"

dataset = load_dataset("json", data_files={"data": SRC})["data"]
dataset = dataset.map(lambda x: {"type": detect_type(x["filename"])})

cryptol = dataset.filter(lambda x: x["type"] == "cryptol")
saw = dataset.filter(lambda x: x["type"] == "saw")
text = dataset.filter(lambda x: x["type"] == "text")

cryptol_split = cryptol.train_test_split(test_size=0.1, seed=42)
saw_split = saw.train_test_split(test_size=0.1, seed=42)
text_split = text.train_test_split(test_size=0.1, seed=42)

train_ds = concatenate_datasets([cryptol_split["train"], saw_split["train"], text_split["train"]])
eval_ds = concatenate_datasets([cryptol_split["test"], saw_split["test"], text_split["test"]])

print(train_ds[0].keys())
assert DATA_COLUMN in train_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"

print(eval_ds[0].keys())
assert DATA_COLUMN in eval_ds.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"

print(len(train_ds), len(eval_ds))

def chars_token_ratio(dataset, tokenizer, data_column, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """

    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        total_characters += len(example[data_column])
        total_tokens += len(tokenizer(example[data_column]).tokens())

    return total_characters / total_tokens


chars_per_token = chars_token_ratio(train_ds, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

Filter:   0%|          | 0/512 [00:00<?, ? examples/s]

Filter:   0%|          | 0/512 [00:00<?, ? examples/s]

dict_keys(['filename', 'filetype', 'content', 'variant', 'input_ids', 'n_tokens', 'set', 'type'])
dict_keys(['filename', 'filetype', 'content', 'variant', 'input_ids', 'n_tokens', 'set', 'type'])
460 52


100%|██████████| 400/400 [00:01<00:00, 272.17it/s]

The character to token ratio of the dataset is: 2.39


In [ ]:
from model.dataset.util import *

In [ ]:
from model.dataset.constantLengthDataset import ConstantLengthDataset
import torch

def make_overlapping_chunks(texts, tokenizer, seq_length=2048, stride=1024):
  tokens = tokenizer(texts, add_special_tokens=False)["input_ids"]
  flat = [token for seq in tokens for token in seq]
  chunks = []

  for i in range(0, len(flat) - seq_length, stride):
    chunk = flat[i : i + seq_length]
    chunks.append(tokenizer.decode(chunk))

  return chunks

overlap_train = make_overlapping_chunks(
    [item[DATA_COLUMN] for item in train_ds],
    tokenizer,
    seq_length=SEQ_LENGTH,
    stride=SEQ_LENGTH // 2
)

overlap_eval = make_overlapping_chunks(
    [item[DATA_COLUMN] for item in eval_ds],
    tokenizer,
    seq_length=SEQ_LENGTH,
    stride=SEQ_LENGTH // 2
)

train_ds_overlap = Dataset.from_dict({DATA_COLUMN: overlap_train})
eval_ds_overlap = Dataset.from_dict({DATA_COLUMN: overlap_eval})


train_dataset = ConstantLengthDataset(
        tokenizer,
        train_ds_overlap,
        # train_ds,
        infinite=True,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)
eval_dataset = ConstantLengthDataset(
        tokenizer,
        eval_ds_overlap,
        # eval_ds,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

load_in_8bit = False

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

base = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config,
        dtype=torch.bfloat16,
        device_map="auto",
        use_cache=False,  # We will be using gradient checkpointing
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
)

model = prepare_model_for_kbit_training(base)

config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00001-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

model-00014-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00015-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00016-of-00016.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
blk = model.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
# target_modules = ["q_proj","v_proj","up_proj","down_proj"]
target_modules = ["q_proj","k_proj","v_proj","o_proj", "gate"]

ATTN: Qwen3MoeAttention(
  (q_proj): Linear4bit(in_features=2048, out_features=4096, bias=False)
  (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
  (o_proj): Linear4bit(in_features=4096, out_features=2048, bias=False)
  (q_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
  (k_norm): Qwen3MoeRMSNorm((128,), eps=1e-06)
)
MLP: Qwen3MoeSparseMoeBlock(
  (gate): Linear4bit(in_features=2048, out_features=128, bias=False)
  (experts): ModuleList(
    (0-127): 128 x Qwen3MoeMLP(
      (gate_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (up_proj): Linear4bit(in_features=2048, out_features=768, bias=False)
      (down_proj): Linear4bit(in_features=768, out_features=2048, bias=False)
      (act_fn): SiLUActivation()
    )
  )
)


In [ ]:
# Set up lora
from peft import PeftModel

peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

# model = PeftModel.from_pretrained(base, "tam2003/peft-FT-3-Coder-30b") # If your just trying to test the model.
model = get_peft_model(base, peft_config) # If you want to continue training.

model.print_trainable_parameters()

trainable params: 15,040,512 || all params: 30,547,163,136 || trainable%: 0.0492


In [ ]:
train_ds.start_iteration = 0

training_args = TrainingArguments(
    output_dir=f"tam1444AH/{OUTPUT_DIR}",
    dataloader_drop_last=True,
    eval_strategy="steps",
    save_strategy="steps",
    report_to=["tensorboard"],
    max_steps=MAX_STEPS,
    eval_steps=EVAL_FREQ,
    save_steps=SAVE_FREQ,
    logging_steps=LOG_FREQ,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=NUM_WARMUP_STEPS,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=True,
    fp16=FP16,
    bf16=BF16,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=True,
    include_tokens_per_second=True,
    resume_from_checkpoint=False,
    dataloader_num_workers=4,
    dataloader_prefetch_factor=2,
)

print(f"Training samples: {len(train_ds)}")
print(f"Sample text:\n\n{train_ds[0][DATA_COLUMN][:400]}")

Training samples: 460
Sample text:

/**
 * @copyright Galois, Inc.
 * @author John Christensen <jchristensen@galois.com>
 *
 * Parameters are taken from Table 3.1 of the Simon documentation.
 *
 */

module Primitive::Symmetric::Cipher::Block::Simon::Instantiations::Simon64_96 =
  Primitive::Symmetric::Cipher::Block::Simon::Specification where
    type n = 32
    type m = 3
    type T = 42
    type j = 2



In [ ]:
import math

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

print("Training...")
trainer.train(resume_from_checkpoint=False)

eval_results = trainer.evaluate()
eval_loss = eval_results["eval_loss"]
perplexity = math.exp(eval_loss)
print(f"Eval loss = {eval_loss:.2f}, Perplexity = {perplexity:.2f}")

/tmp/ipython-input-1014676409.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 151643}.


Training...


Casting fp32 inputs back to torch.bfloat16 for flash-attn compatibility.


Step,Training Loss,Validation Loss
25,1.041700,1.457827
50,0.902600,1.418585
75,0.924000,1.404917
100,0.892400,1.397537
125,0.786600,1.395952
150,0.778600,1.394875
175,0.853300,1.394318
200,0.815700,1.392747


Eval loss = 1.39, Perplexity = 4.03


In [ ]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...der-30b/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...963931.3beaf68aad32.977.0: 100%|##########| 12.7kB / 12.7kB            

  ...-Coder-30b/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...adapter_model.safetensors:  70%|######9   | 41.9MB / 60.2MB            

  ...980338.3beaf68aad32.977.1: 100%|##########|   359B /   359B            

CommitInfo(commit_url='https://huggingface.co/tam2003/peft-FT-3-Coder-30b/commit/debf44af6745e5ca1de75159c49b0246c42c0b9e', commit_message='End of training', commit_description='', oid='debf44af6745e5ca1de75159c49b0246c42c0b9e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tam2003/peft-FT-3-Coder-30b', endpoint='https://huggingface.co', repo_type='model', repo_id='tam2003/peft-FT-3-Coder-30b'), pr_revision=None, pr_num=None)

In [ ]:
!pip install -q transformers peft
!pip install -q transformers peft bitsandbytes accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Base model load
base = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-Coder-30B-A3B-Instruct",
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA fine-tuned adapter on top of it
model = PeftModel.from_pretrained(base, "tam2003/peft-FT-3-Coder-30b")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Coder-30B-A3B-Instruct", trust_remote_code=True)
model.eval()


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/60.2M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3MoeForCausalLM(
      (model): Qwen3MoeModel(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-47): 48 x Qwen3MoeDecoderLayer(
            (self_attn): Qwen3MoeAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
from transformers import pipeline
import gc, torch
gc.collect()
torch.cuda.empty_cache()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="balanced_low_0",
    dtype="bfloat16",
)

prompt = """
Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input block and returns a 256-bit digest."
"""
result = pipe(prompt, max_new_tokens=300)
print(result[0]["generated_text"])

prompts = [
    'Translate this English spec into Cryptol:\n\n"A function xor_inverse that proves (a XOR b) XOR b == a"',
    'Translate this English spec into Cryptol:\n\n"A function sha256_hash that takes a 512-bit input and returns a 256-bit digest"',
    'Translate this English spec into Cryptol:\n\n"Create a property proving that addition is commutative for 8-bit words."'
]

for p in prompts:
    print("\n=== Prompt ===")
    print(p)
    inputs = tokenizer(p, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    print("\n--- Generated Cryptol ---")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Device set to use cuda:0



Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input block and returns a 256-bit digest."
module SHA256 where

sha256_hash : [512] -> [256]
sha256_hash = error "unimplemented"


=== Prompt ===
Translate this English spec into Cryptol:

"A function xor_inverse that proves (a XOR b) XOR b == a"

--- Generated Cryptol ---
Translate this English spec into Cryptol:

"A function xor_inverse that proves (a XOR b) XOR b == a"

=== Prompt ===
Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input and returns a 256-bit digest"

--- Generated Cryptol ---
Translate this English spec into Cryptol:

"A function sha256_hash that takes a 512-bit input and returns a 256-bit digest"

=== Prompt ===
Translate this English spec into Cryptol:

"Create a property proving that addition is commutative for 8-bit words."

--- Generated Cryptol ---
Translate this English spec into Cryptol:

"Create a property proving that addition i